# Snapshot Serengeti Inception-V3

## Disclaimer
This uses the pre-built and open source inception v3 model as a baseline.
For more information see https://www.tensorflow.org/tutorials/image_recognition#usage_with_python_api.
You can also find the original source code that is modified here at https://github.com/tensorflow/models

In [1]:
# Basic Data Imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Google Inception Imports
from six.moves import urllib
import os.path
import sys
import tarfile
import re

In [3]:
# Constants
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
DEST_DIRECTORY = "/vagrant/models"
NUM_TOP_PREDICTIONS = 5

# Global State
GRAPH_STARTED = False

In [4]:
class NodeLookup(object):
  """Converts integer node ID's to human readable labels."""

  def __init__(self,
               label_lookup_path=None,
               uid_lookup_path=None):
    if not label_lookup_path:
      label_lookup_path = os.path.join(
          DEST_DIRECTORY, 'imagenet_2012_challenge_label_map_proto.pbtxt')
    if not uid_lookup_path:
      uid_lookup_path = os.path.join(
          DEST_DIRECTORY, 'imagenet_synset_to_human_label_map.txt')
    self.node_lookup = self.load(label_lookup_path, uid_lookup_path)

  def load(self, label_lookup_path, uid_lookup_path):
    """Loads a human readable English name for each softmax node.

    Args:
      label_lookup_path: string UID to integer node ID.
      uid_lookup_path: string UID to human-readable string.

    Returns:
      dict from integer node ID to human-readable string.
    """
    if not tf.gfile.Exists(uid_lookup_path):
      tf.logging.fatal('File does not exist %s', uid_lookup_path)
    if not tf.gfile.Exists(label_lookup_path):
      tf.logging.fatal('File does not exist %s', label_lookup_path)

    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = tf.gfile.GFile(uid_lookup_path).readlines()
    uid_to_human = {}
    p = re.compile(r'[n\d]*[ \S,]*')
    for line in proto_as_ascii_lines:
      parsed_items = p.findall(line)
      uid = parsed_items[0]
      human_string = parsed_items[2]
      uid_to_human[uid] = human_string

    # Loads mapping from string UID to integer node ID.
    node_id_to_uid = {}
    proto_as_ascii = tf.gfile.GFile(label_lookup_path).readlines()
    for line in proto_as_ascii:
      if line.startswith('  target_class:'):
        target_class = int(line.split(': ')[1])
      if line.startswith('  target_class_string:'):
        target_class_string = line.split(': ')[1]
        node_id_to_uid[target_class] = target_class_string[1:-2]

    # Loads the final mapping of integer node ID to human-readable string
    node_id_to_name = {}
    for key, val in node_id_to_uid.items():
      if val not in uid_to_human:
        tf.logging.fatal('Failed to locate: %s', val)
      name = uid_to_human[val]
      node_id_to_name[key] = name

    return node_id_to_name

  def id_to_string(self, node_id):
    if node_id not in self.node_lookup:
      return ''
    return self.node_lookup[node_id]

In [5]:
# Download Google Incpetion v3 Model
# Based on https://www.tensorflow.org/tutorials/image_recognition#usage_with_python_api

def maybe_download_and_extract():
  """Download and extract model tar file."""
  if not os.path.exists(DEST_DIRECTORY):
    os.makedirs(DEST_DIRECTORY)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(DEST_DIRECTORY, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (
          filename, float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(DEST_DIRECTORY)


In [9]:
def create_graph():
  global GRAPH_STARTED
  if GRAPH_STARTED:
    return

  GRAPH_STARTED = True
  """Creates a graph from saved GraphDef file and returns a saver."""
  # Creates graph from saved graph_def.pb.
  print('starting create graph')
  with tf.gfile.FastGFile(os.path.join(
      DEST_DIRECTORY, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
  print('done creating graph')

In [14]:
def run_inference_on_image(image):
  """Runs inference on an image.

  Args:
    image: Image file name.

  Returns:
    Nothing
  """
  print('loading file')
  if not tf.gfile.Exists(image):
    tf.logging.fatal('File does not exist %s', image)
  image_data = tf.gfile.FastGFile(image, 'rb').read()

  # Creates graph from saved GraphDef.
  print('creating graph')
  create_graph()

  print('starting session')
  with tf.Session() as sess:
    # Some useful tensors:
    # 'softmax:0': A tensor containing the normalized prediction across
    #   1000 labels.
    # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
    #   float description of the image.
    # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
    #   encoding of the image.
    # Runs the softmax tensor by feeding the image_data as input to the graph.
    softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    print('running session')
    predictions = sess.run(softmax_tensor,
                           {'DecodeJpeg/contents:0': image_data})
    print('run ended, with predictions: ' + str(predictions))
    predictions = np.squeeze(predictions)

    print('starting node lookup')
    # Creates node ID --> English string lookup.
    node_lookup = NodeLookup()
    print('end node lookup')

    print('start sort')
    top_k = predictions.argsort()[-NUM_TOP_PREDICTIONS:][::-1]
    for node_id in top_k:
      human_string = node_lookup.id_to_string(node_id)
      score = predictions[node_id]
      print('%s (score = %.5f)' % (human_string, score))

In [15]:
# Run classification on snapshot data
maybe_download_and_extract()
image = '/vagrant/data/snapshot/Elephant/S5_D02_R3_IMAG0860.JPG'
run_inference_on_image(image)

loading file
creating graph
starting session
running session
run ended, with predictions: [[  1.19376884e-04   3.70689995e-05   3.15405603e-04 ...,   1.19377401e-04
    1.19376149e-04   1.19376149e-04]]
starting node lookup
end node lookup
start sort
seashore, coast, seacoast, sea-coast (score = 0.25032)
lakeside, lakeshore (score = 0.07298)
plow, plough (score = 0.05792)
worm fence, snake fence, snake-rail fence, Virginia fence (score = 0.03367)
ear, spike, capitulum (score = 0.02446)
